# 🧠 Waste Classification using MobileNetV2

This notebook trains a real AI model to classify waste into:
- Plastic
- Metal
- Organic
- E-waste (optional if added)

It uses MobileNetV2 with image augmentation for good accuracy.


In [1]:
# ✅ Step 1: Install dependencies
!pip install tensorflow --quiet

You should consider upgrading via the 'C:\Users\vanee\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import os
root = "waste_dataset"
for folder in os.listdir(root):
    folder_path = os.path.join(root, folder)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder}")

Processing folder: metal
Processing folder: organic
Processing folder: plastic


In [6]:
# ✅ Step 3: Setup train/val datasets with augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    "waste_dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    "waste_dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 820 images belonging to 3 classes.
Found 203 images belonging to 3 classes.


In [9]:
# ✅ Step 4: Load MobileNetV2 and train the model
import tensorflow as tf
base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224,224,3), weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ✅ Add early stopping
cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=[cb]
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6201 - loss: 0.8514

c:\Users\vanee\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.6240 - loss: 0.8435 - val_accuracy: 0.8276 - val_loss: 0.4987
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.8858 - loss: 0.3044 - val_accuracy: 0.8325 - val_loss: 0.4286
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9025 - loss: 0.2355 - val_accuracy: 0.7685 - val_loss: 0.5338
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.9062 - loss: 0.2301 - val_accuracy: 0.8227 - val_loss: 0.4382
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.9369 - loss: 0.1540 - val_accuracy: 0.7980 - val_loss: 0.4662


In [16]:
# ✅ Step 5: Save the model
model.save("waste_model.h5")
print("waste_model.h5")

waste_model.h5
